In [27]:
import pandas as pd
import requests
import json,math
from datetime import datetime as dt
import threading
import time
from datetime import timedelta
import numpy as np
import re

# Converts Epoch time to readable format
def local_time(epoch_time):
    return time.strftime('%d-%m-%Y %H:%M:%S', time.localtime(int(str(epoch_time)[0:10])))

# Function to extract data from url
def data_from_url (urls):
  #start = dt.datetime.now()
  df = pd.DataFrame() 
    
  # url1 to store urls facing error_code 504 : Api limit exceeds  
  url1 = list()
  
  for url in urls:
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
            data = pd.DataFrame(json.loads(response.text), columns=['time', 'low', 'high', 'open', 'close', 'volume'])
            #if len(data.index) == 0:
             #  print('XOXO')
              # Can print urls where for which coindcx doesn't have data
              # print(str(len(data.index))+ " " + url + '\n')
            df = df.append(data)
    
    # when error code = 504, Again call API request
    elif response.status_code == 504:
        #print(f'{response.status_code} -- Error in 504 ' + url + '\n')
        # Add sleep for cases where Limit exceed error occurs. Saves Unnessary sleep used on above if !!
        time.sleep(0.1)    
        response1 = requests.get(url)
        if response1.status_code == 200:
            data = pd.DataFrame(json.loads(response1.text), columns=['time', 'low', 'high', 'open', 'close', 'volume','date'])
            df = df.append(data)
        else: # No Need for this part : Just here for safer side.
            print("Did not receieve OK response from Coinbase API:" + url)
            print(f'{response.status_code}: Error in else of 504')
    else:
        print("Did not receieve OK response from Coinbase API:" + url)
        print(f'{response.status_code}: Error in else')
    end = dt.now()
  
  # Very less chance of occuring this !!  
  if len(url1) > 0:
    data_from_url(url1)
  
  #print('END of urls')
  # Convert date to readable format  
  df['date'] = df.apply(lambda x: local_time(x['time']), axis=1)  
  return df      

# Function generated all urls for given currency
url = "https://public.coindcx.com/market_data/candles?pair=B-BTC_USDT&interval=1m" 
def generate_urls_from_symbol_name(range_end,symbol,epoch_startDate,epoch_endDate):
    # List to store all url
    urls = []
    interval = '5m'
    for i in range(0,range_end):
        #minutes = 1000*interval
        url_endDate = int((dt.fromtimestamp(epoch_startDate) + timedelta(minutes =5000)).timestamp())
        
        if epoch_endDate > url_endDate:
            urls.append(f"https://public.coindcx.com/market_data/candles?pair={symbol}&interval={interval}&startTime={int(epoch_startDate)}000&endTime={int(url_endDate)}000&limit=1000")
            epoch_startDate = url_endDate
        elif epoch_endDate < url_endDate:
            urls.append(f"https://public.coindcx.com/market_data/candles?pair={symbol}&interval={interval}&startTime={int(epoch_startDate)}000&endTime={int(epoch_endDate)}000&limit=1000")

    return urls

def fetch_daily_data(pair,range_end,epoch_startDate,epoch_endDate,n):
  for p in pair:  
    urls = generate_urls_from_symbol_name(range_end,p,epoch_startDate,epoch_endDate)
    data = data_from_url(urls)
    # if we failed to get any data, print an error...otherwise write the file
    if data is None:
            print("Did not return any data from Coinbase for this symbol:"+ p  )
    else:
            data.to_csv(f'CoinDCX_{p}_dailydata.csv', index=False)
            # n is worker number
            print(f'CoinDCX_{p}_dailydata.csv',n)


if __name__ == "__main__":
    # uncomment below code if you want to run against all Coins.
    
    #url = "https://api.coindcx.com/exchange/v1/markets_details"
    #response = requests.get(url).json()
    #pair = []
    #for i in range(len(response)):
     #   if re.search('^B-.*USDT$',response[i]['pair']):
      #      pair.append(response[i]['pair'])
    
    # we set which pair we want to retrieve data for
    #pair1 = ['B-ACA_USDT']
    # Here B- denotes Binance Data, Similarly I-  Denotes Coinbase Data etc. Check API documentation for more info
    
    pair = ['B-ACA_USDT', 'B-DENT_USDT', 'B-REEF_USDT', 'B-CLV_USDT', 'B-SPELL_USDT', 'B-CHESS_USDT', 'B-MITH_USDT', 
            'B-FIRO_USDT','B-BNX_USDT', 'B-MDX_USDT', 'B-NBS_USDT', 'B-SLP_USDT', 'B-CFX_USDT', 'B-AUTO_USDT', 'B-OOKI_USDT', 
            'B-COMP_USDT', 'B-CITY_USDT', 'B-LIT_USDT', 'B-TRB_USDT', 'B-BAND_USDT', 'B-UNI_USDT', 'B-RAD_USDT', 'B-BAR_USDT', 
            'B-RAY_USDT', 'B-XTZ_USDT', 'B-TLM_USDT', 'B-UST_USDT', 'B-CELR_USDT', 'B-POLS_USDT', 'B-BTC_USDT', 'B-FIS_USDT', 
            'B-BAT_USDT', 'B-KNC_USDT', 'B-ENJ_USDT', 'B-DATA_USDT' , 'B-IOTA_USDT', 'B-GTC_USDT', 'B-IMX_USDT', 'B-KSM_USDT', 
            'B-DAR_USDT', 'B-HNT_USDT', 'B-ANC_USDT', 'B-IRIS_USDT', 'B-JOE_USDT' , 'B-ARPA_USDT', 'B-DOCK_USDT', 'B-BEAM_USDT',
            'B-WTC_USDT', 'B-MKR_USDT', 'B-BETA_USDT', 'B-YGG_USDT', 'B-CHZ_USDT', 'B-REQ_USDT', 'B-CTXC_USDT', 'B-POLY_USDT', 
            'B-LTC_USDT', 'B-SUN_USDT', 'B-PNT_USDT', 'B-FIL_USDT', 'B-DASH_USDT', 'B-ARDR_USDT', 'B-GHST_USDT', 'B-ASTR_USDT',
            'B-BTS_USDT', 'B-PERL_USDT', 'B-PSG_USDT', 'B-XVS_USDT', 'B-MFT_USDT', 'B-PUNDIX_USDT', 'B-COS_USDT', 'B-PLA_USDT',
            'B-FLM_USDT', 'B-FIO_USDT', 'B-LPT_USDT', 'B-COTI_USDT', 'B-ONG_USDT', 'B-WOO_USDT', 'B-STX_USDT', 'B-MOVR_USDT', 
            'B-VET_USDT', 'B-DCR_USDT', 'B-FLOW_USDT', 'B-REP_USDT', 'B-FLUX_USDT', 'B-AAVE_USDT', 'B-THETA_USDT', 
            'B-ALICE_USDT', 'B-ADA_USDT', 'B-XEM_USDT', 'B-CAKE_USDT', 'B-ICP_USDT', 'B-CKB_USDT', 'B-ETC_USDT', 'B-STMX_USDT',
            'B-RAMP_USDT', 'B-RGT_USDT', 'B-OXT_USDT', 'B-TKO_USDT', 'B-ACH_USDT', 'B-SOL_USDT', 'B-QTUM_USDT' , 'B-NKN_USDT', 
            'B-WING_USDT', 'B-AION_USDT', 'B-WRX_USDT', 'B-DOGE_USDT', 'B-WIN_USDT', 'B-SRM_USDT', 'B-AKRO_USDT', 'B-AVAX_USDT',
            'B-QNT_USDT', 'B-HIGH_USDT', 'B-SC_USDT', 'B-DEGO_USDT', 'B-DODO_USDT', 'B-CVX_USDT', 'B-WNXM_USDT', 'B-ZRX_USDT', 
            'B-AUDIO_USDT', 'B-RNDR_USDT', 'B-STRAX_USDT', 'B-ALGO_USDT', 'B-XNO_USDT', 'B-FIDA_USDT', 'B-RARE_USDT', 
            'B-XMR_USDT', 'B-TRU_USDT', 'B-WAN_USDT', 'B-NEO_USDT', 'B-ZEC_USDT', 'B-OMG_USDT', 'B-EGLD_USDT', 'B-SNX_USDT', 
            'B-FTM_USDT', 'B-BEL_USDT', 'B-LUNA_USDT', 'B-MTL_USDT', 'B-ANT_USDT', 'B-PERP_USDT', 'B-ERN_USDT', 'B-1INCH_USDT',
            'B-MDT_USDT', 'B-INJ_USDT', 'B-SHIB_USDT', 'B-WAVES_USDT', 'B-KLAY_USDT', 'B-CELO_USDT', 'B-BICO_USDT',
            'B-DGB_USDT', 'B-HARD_USDT', 'B-EOS_USDT', 'B-ZIL_USDT', 'B-CTSI_USDT', 'B-CRV_USDT', 'B-POND_USDT', 'B-HBAR_USDT', 
            'B-TRX_USDT', 'B-BNB_USDT', 'B-DREP_USDT', 'B-PEOPLE_USDT', 'B-UNFI_USDT', 'B-BCH_USDT', 'B-WAXP_USDT', 
            'B-PYR_USDT', 'B-API3_USDT', 'B-ALPACA_USDT', 'B-AMP_USDT', 'B-BAL_USDT', 'B-ANKR_USDT', 'B-OM_USDT', 'B-XLM_USDT',
            'B-TWT_USDT', 'B-MBL_USDT', 'B-IOTX_USDT', 'B-COCOS_USDT', 'B-BNT_USDT', 'B-HOT_USDT', 'B-GALA_USDT', 'B-LRC_USDT',
            'B-RVN_USDT', 'B-DYDX_USDT', 'B-ALPHA_USDT', 'B-YFI_USDT', 'B-FET_USDT', 'B-STORJ_USDT', 'B-SUPER_USDT', 
            'B-NMR_USDT', 'B-ZEN_USDT', 'B-OGN_USDT', 'B-YFII_USDT', 'B-TFUEL_USDT', 'B-KEY_USDT', 'B-ICX_USDT', 'B-GLMR_USDT',
            'B-STPT_USDT', 'B-EPS_USDT', 'B-DUSK_USDT', 'B-UMA_USDT', 'B-SUSHI_USDT', 'B-NEAR_USDT', 'B-KP3R_USDT', 
            'B-ONE_USDT', 'B-OCEAN_USDT', 'B-MIR_USDT', 'B-BLZ_USDT', 'B-JST_USDT', 'B-KAVA_USDT', 'B-AXS_USDT', 'B-SYS_USDT',
            'B-ATOM_USDT', 'B-CVC_USDT', 'B-ETH_USDT', 'B-IOST_USDT', 'B-NULS_USDT', 'B-GTO_USDT', 'B-RUNE_USDT', 'B-LSK_USDT',
            'B-RLC_USDT', 'B-LINK_USDT', 'B-GRT_USDT', 'B-SAND_USDT', 'B-MASK_USDT', 'B-ROSE_USDT', 'B-CHR_USDT', 'B-ILV_USDT', 
            'B-SXP_USDT', 'B-XRP_USDT', 'B-MANA_USDT', 'B-TROY_USDT', 'B-FTT_USDT', 'B-REN_USDT', 'B-HIVE_USDT', 'B-ALCX_USDT',
            'B-MATIC_USDT', 'B-GNO_USDT', 'B-SKL_USDT', 'B-MINA_USDT', 'B-DOT_USDT', 'B-ONT_USDT', 'B-BTTC_USDT', 'B-TUSD_USDT',
            'B-BUSD_USDT', 'B-MC_USDT', 'B-FUN_USDT', 'B-UTK_USDT', 'B-USDP_USDT', 'B-ANY_USDT', 'B-TOMO_USDT', 'B-USDC_USDT']
    epoch_startDate = dt(2021,6,1).timestamp()
    epoch_endDate = dt(2022,3,12).timestamp()
    # 60*1000*interval
    range_end = math.ceil((epoch_endDate-epoch_startDate)/(60*1000*5))
    print(range_end, len(pair))
    # Declare number of threads
    parallel_threads = 10
    #Splitting Pair files for each thread
    parallel_arrays = np.array_split(pair, parallel_threads)
    workers = list()
  
    start = dt.now()
    #Process Pair Files parallely
    for k in range(parallel_threads):
        # Implemented MultiThreading to process 3 threads in parallel
        wp = threading.Thread(target=fetch_daily_data, name='worker%d' % (k + 1), args=(list(parallel_arrays[k]),range_end,epoch_startDate,epoch_endDate,k+1,))
        workers.append(wp)
        wp.start()
   
    #Waitng for threads to complete
    for wp in workers:
        wp.join()
    end = dt.now()
    # Print total Time to extract Data
    print('END of program')
    print(start, end , end-start)

82 249
504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-TLM_USDT&interval=5m&startTime=1622823600000&endTime=1623123600000&limit=1000

504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-EOS_USDT&interval=5m&startTime=1622823600000&endTime=1623123600000&limit=1000

END of urls
CoinDCX_B-ACA_USDT_dailydata.csv 1
END of urls
CoinDCX_B-ONG_USDT_dailydata.csv 4
END of urls
CoinDCX_B-QTUM_USDT_dailydata.csv 5
END of urls
CoinDCX_B-YGG_USDT_dailydata.csv 3
END of urls
CoinDCX_B-SXP_USDT_dailydata.csv 10
END of urls
CoinDCX_B-BNT_USDT_dailydata.csv 8
504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-TLM_USDT&interval=5m&startTime=1643527200000&endTime=1643827200000&limit=1000

504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-KP3R_USDT&interval=5m&startTime=1642927200000&endTime=1643227200000&limit=1000

504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-XMR_USDT&interval=5m&s

END of urls
CoinDCX_B-AVAX_USDT_dailydata.csv 5
END of urls
CoinDCX_B-FET_USDT_dailydata.csv 8
END of urls
END of urls
CoinDCX_B-GNO_USDT_dailydata.csv 10
CoinDCX_B-ENJ_USDT_dailydata.csv 2
END of urls
CoinDCX_B-NBS_USDT_dailydata.csv 1
END of urls
CoinDCX_B-PEOPLE_USDT_dailydata.csv 7
END of urls
CoinDCX_B-ARDR_USDT_dailydata.csv 3
END of urls
CoinDCX_B-LUNA_USDT_dailydata.csv 6
END of urls
CoinDCX_B-QNT_USDT_dailydata.csv 5
504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-THETA_USDT&interval=5m&startTime=1644127200000&endTime=1644427200000&limit=1000

END of urls
CoinDCX_B-STORJ_USDT_dailydata.csv 8
END of urls
CoinDCX_B-THETA_USDT_dailydata.csv 4
END of urls
CoinDCX_B-ATOM_USDT_dailydata.csv 9
END of urls
END of urls
CoinDCX_B-UNFI_USDT_dailydata.csv 7
CoinDCX_B-DATA_USDT_dailydata.csv 2
END of urls
CoinDCX_B-SLP_USDT_dailydata.csv 1
END of urls
CoinDCX_B-SKL_USDT_dailydata.csv 10
END of urls
CoinDCX_B-GHST_USDT_dailydata.csv 3
END of urls
CoinDCX_B-HIGH_USD

END of urls
CoinDCX_B-FUN_USDT_dailydata.csv 10
END of urls
CoinDCX_B-COS_USDT_dailydata.csv 3
504 -- Error in 504 https://public.coindcx.com/market_data/candles?pair=B-IRIS_USDT&interval=5m&startTime=1632723600000&endTime=1633023600000&limit=1000

END of urls
CoinDCX_B-ICX_USDT_dailydata.csv 8
END of urls
CoinDCX_B-RNDR_USDT_dailydata.csv 5
END of urls
CoinDCX_B-RLC_USDT_dailydata.csv 9
END of urls
CoinDCX_B-OM_USDT_dailydata.csv 7
END of urls
END of urls
CoinDCX_B-RGT_USDT_dailydata.csv 4
CoinDCX_B-WAVES_USDT_dailydata.csv 6
END of urls
CoinDCX_B-BAND_USDT_dailydata.csv 1
END of urls
CoinDCX_B-IRIS_USDT_dailydata.csv 2
END of urls
CoinDCX_B-UTK_USDT_dailydata.csv 10
END of urls
CoinDCX_B-PLA_USDT_dailydata.csv 3
END of urls
CoinDCX_B-GLMR_USDT_dailydata.csv 8
END of urls
CoinDCX_B-STRAX_USDT_dailydata.csv 5
END of urls
END of urls
CoinDCX_B-LINK_USDT_dailydata.csv 9
CoinDCX_B-OXT_USDT_dailydata.csv 4
END of urls
CoinDCX_B-XLM_USDT_dailydata.csv 7
END of urls
CoinDCX_B-UNI_USDT_dailyd

In [28]:
import glob
# 1. list all text files in the directory
rel_filepaths = glob.glob("*.csv")

# 2. (optional) create a function to read the number of rows in a file
def count_rows(filepath):
  res = 0
  f = open(filepath, 'r')
  res = len(f.readlines())
  f.close()
  return res

# 3. iterate over your files and use the count_row function
ttotal=0
for filepath in rel_filepaths:  
  filecount = count_rows(filepath) 
  print(f'{filepath},{filecount}')
  ttotal += filecount
print(f'Total count:0{ttotal}')


CoinDCX_B-1INCH_USDT_dailydata.csv,81333
CoinDCX_B-AAVE_USDT_dailydata.csv,81299
CoinDCX_B-ACA_USDT_dailydata.csv,12925
CoinDCX_B-ACH_USDT_dailydata.csv,16141
CoinDCX_B-ADA_USDT_dailydata.csv,80445
CoinDCX_B-AION_USDT_dailydata.csv,80611
CoinDCX_B-AKRO_USDT_dailydata.csv,81212
CoinDCX_B-ALCX_USDT_dailydata.csv,25290
CoinDCX_B-ALGO_USDT_dailydata.csv,81495
CoinDCX_B-ALICE_USDT_dailydata.csv,81315
CoinDCX_B-ALPACA_USDT_dailydata.csv,58154
CoinDCX_B-ALPHA_USDT_dailydata.csv,80416
CoinDCX_B-AMP_USDT_dailydata.csv,28837
CoinDCX_B-ANC_USDT_dailydata.csv,12002
CoinDCX_B-ANKR_USDT_dailydata.csv,81334
CoinDCX_B-ANT_USDT_dailydata.csv,80981
CoinDCX_B-ANY_USDT_dailydata.csv,27199
CoinDCX_B-API3_USDT_dailydata.csv,12756
CoinDCX_B-ARDR_USDT_dailydata.csv,78135
CoinDCX_B-ARPA_USDT_dailydata.csv,81478
CoinDCX_B-ASTR_USDT_dailydata.csv,2839
CoinDCX_B-ATOM_USDT_dailydata.csv,81454
CoinDCX_B-AUDIO_USDT_dailydata.csv,80364
CoinDCX_B-AUTO_USDT_dailydata.csv,78295
CoinDCX_B-AVAX_USDT_dailydata.csv,81504
Co

CoinDCX_B-TKO_USDT_dailydata.csv,81432
CoinDCX_B-TLM_USDT_dailydata.csv,14387
CoinDCX_B-TOMO_USDT_dailydata.csv,81318
CoinDCX_B-TRB_USDT_dailydata.csv,81222
CoinDCX_B-TROY_USDT_dailydata.csv,79773
CoinDCX_B-TRU_USDT_dailydata.csv,60441
CoinDCX_B-TRX_USDT_dailydata.csv,81336
CoinDCX_B-TUSD_USDT_dailydata.csv,81359
CoinDCX_B-TWT_USDT_dailydata.csv,81238
CoinDCX_B-UMA_USDT_dailydata.csv,80511
CoinDCX_B-UNFI_USDT_dailydata.csv,81329
CoinDCX_B-UNI_USDT_dailydata.csv,81314
CoinDCX_B-USDC_USDT_dailydata.csv,81502
CoinDCX_B-USDP_USDT_dailydata.csv,37599
CoinDCX_B-UST_USDT_dailydata.csv,21183
CoinDCX_B-UTK_USDT_dailydata.csv,80996
CoinDCX_B-VET_USDT_dailydata.csv,81468
CoinDCX_B-WAN_USDT_dailydata.csv,81253
CoinDCX_B-WAVES_USDT_dailydata.csv,81502
CoinDCX_B-WAXP_USDT_dailydata.csv,30865
CoinDCX_B-WING_USDT_dailydata.csv,78761
CoinDCX_B-WIN_USDT_dailydata.csv,81456
CoinDCX_B-WNXM_USDT_dailydata.csv,74352
CoinDCX_B-WOO_USDT_dailydata.csv,9129
CoinDCX_B-WRX_USDT_dailydata.csv,81449
CoinDCX_B-WTC_U